# The GC calculator

I downloaded the genome of *Utricularia gibba* (humped bladderwort) from [NCBI](https://www.ncbi.nlm.nih.gov/data-hub/genome/GCA_002189035.1/) and decompressed it. The genome has one file per chromosome (e.g. "chr1.fna"). You can download the data for yourself to take a look!

I have two goals for this code:
* Get the label of the **chromosome** with the largest percentage of GC content
* Calculate the GC percentage of the whole genome

However, I have a couple of issues. Can you help me fix my code?



In [2]:
from pathlib import Path

ok, let me make sure we have the .fna files

For my first goal, I have to ignore the mitochondrial, plastid and unplaced genomic data (which all end in .scaf.fna)

In [3]:
ls

 chr1.fna   chrMT.unlocalized.scaf.fna    'GC_calculator fixed.ipynb'
 chr2.fna   chrPltd.unlocalized.scaf.fna   GC_calculator.ipynb
 chr3.fna   fix_GC_content.md             'GCF_001433935.1 O sativa.zip'
 chr4.fna   GCA_002189035.1.zip            unplaced.scaf.fna


There's something funny with my calculation... it should match the ones in NCBI

Can you figure out what's wrong?

In [4]:
def gc_calculator(data):
    """Calculates the total GC content

    Parameters
    ---

    data: dict
        A dictionary with key=records' labels, values=sequences

    Returns:
    ---

    record_max_gc: str
        The label of the sequence with the highest gc content
    total_gc: float
        The percentage of gc content for all data
    """

    # counters for total gc, atgc
    gc = 0
    atgc = 0

    # counters for gc, atgc per record
    record_gc = 0
    record_atgc = 0

    label_max_gc = "" # label of sequence with highest gc percentage
    max_gc_record = 0 # keeps the highest gc percentage per record

    # get both key and value
    for label, sequence in data.items():
        atgc += len(sequence)
        record_gc = 0
        
        for char in sequence:
            if char.lower() in {"g", "c"}:
                gc += 1
                record_gc += 1
            else:
                atgc += 1

        # Substitute label_max_gc if better gc found
        if record_gc > max_gc_record:
            max_gc_record = record_gc
            label_max_gc = label
        

    return label_max_gc, gc/atgc

Parsing took more than half an hour on my machine, but a friend of mine can do this in a couple of seconds! How can I speed things up? If I do this on a larger genome, it will never end!

In [5]:
def parse_file(filename, data):
    """Parses a fasta file with a single record

    Parameters:
    ---

    filename: Path
        The path to a fasta file

    data: dict
        A dictionary where label, sequence will be stored

    Returns:
    ---
    
    data: dict
        The updated dictionary
    """

    label = ""

    with open(filename, "r") as f:
        all_lines = f.readlines()

    # First line is the label. Also, remove '>' symbol
    label = all_lines[0][1:]

    # label shouldn't be repeated
    data[label] = ""
    for line in all_lines[1:]:
        data[label] += line

    return data

In [6]:
def main():
    data = {}

    # obtain the paths to every file with .fna extension
    print("Parsing fasta files")
    for fasta in Path("./").glob("*.fna"):
        print(fasta.stem)
        data = parse_file(fasta, data)

    # get the gc numbers
    print("\nCalculating gc percentage values...\n")
    record_max_gc, total_gc_percentage = gc_calculator(data)

    # Print results:
    print(f"Sequence with max GC percentage: {record_max_gc}")
    print(f"Genome GC content: {total_gc_percentage}")

In [7]:
if __name__ == "__main__":
    main()

Parsing fasta files
chr1
chr3
chrMT.unlocalized.scaf
chrPltd.unlocalized.scaf
unplaced.scaf
